In [30]:
import carla 
import math 
import random 
import time 
import numpy as np
import cv2
import open3d as o3d #i sensori restituiscono array di punti 3d
from matplotlib import cm

# connessione al client, spawn e blueprint
client = carla.Client('localhost', 2000)
world = client.get_world()
bp_lib = world.get_blueprint_library() 
spawn_points = world.get_map().get_spawn_points() 

In [31]:
# Add vehicle
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020') 
vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[79])

In [32]:
# pov spettatore su auto
spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform)

# genera 10 auto casuali
for i in range(10): 
    vehicle_bp = random.choice(bp_lib.filter('vehicle')) 
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))    
for v in world.get_actors().filter('*vehicle*'): 
    v.set_autopilot(True) 

In [33]:
# settaggio range di colori per lidar and radar in base alla distanza e velocità
VIRIDIS = np.array(cm.get_cmap('plasma').colors)
VID_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])

COOL_RANGE = np.linspace(0.0, 1.0, VIRIDIS.shape[0])
COOL = np.array(cm.get_cmap('winter')(COOL_RANGE))
COOL = COOL[:,:3]

def add_open3d_axis(vis):
    """Add a small 3D axis on Open3D Visualizer"""
    axis = o3d.geometry.LineSet()
    axis.points = o3d.utility.Vector3dVector(np.array([
        [0.0, 0.0, 0.0],
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]]))
    axis.lines = o3d.utility.Vector2iVector(np.array([
        [0, 1],
        [0, 2],
        [0, 3]]))
    axis.colors = o3d.utility.Vector3dVector(np.array([
        [1.0, 0.0, 0.0],
        [0.0, 1.0, 0.0],
        [0.0, 0.0, 1.0]]))
    vis.add_geometry(axis)

In [34]:
# LIDAR e RADAR callbacks
#lidar cattura come eventi nuvole di punti le variazioni di intensità di colore rilevati durante il movimento dell'auto
def lidar_callback(point_cloud, point_list):
    """Inizializza una nuvola di punti di intensità di colori che verrà elaborata da Open3D"""
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
    data = np.reshape(data, (int(data.shape[0] / 4), 4)) #reshaping array composto da 4 colonne: 3 per x,y,z e la 4 è la distanza dal sensore

    #prende l'ultima colonna ovvero la distanza e la colora in bae a un range di colori
    intensity = data[:, -1] 
    intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004 * 100))
    int_color = np.c_[
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 0]),
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 1]),
        np.interp(intensity_col, VID_RANGE, VIRIDIS[:, 2])]

    points = data[:, :-1] #prende le prime tre colonne contenenti x y z per generare una nuvola di punti

    points[:, :1] = -points[:, :1] #inverte asse y per compatibilità con UE
    
    #memorizza i punti in vettori 3d per visualizzazione a video
    point_list.points = o3d.utility.Vector3dVector(points)
    #colori dei punti in base alla distanza
    point_list.colors = o3d.utility.Vector3dVector(int_color)
    
#non ho ancora ben capito come differisce da lidar, forse cattura le differnze di intesità ad area..?
def radar_callback(data, point_list):
    radar_data = np.zeros((len(data), 4))

    #rilevazione prossimità rispetto a x y z
    for i, detection in enumerate(data):
        x = detection.depth * math.cos(detection.altitude) * math.cos(detection.azimuth)
        y = detection.depth * math.cos(detection.altitude) * math.sin(detection.azimuth)
        z = detection.depth * math.sin(detection.altitude)

        #velocità di rilevazione
        radar_data[i, :] = [x, y, z, detection.velocity]

    #colorazione della velocità definita in un ragne
    intensity = np.abs(radar_data[:, -1])
    intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004 * 100))
    int_color = np.c_[
        np.interp(intensity_col, COOL_RANGE, COOL[:, 0]),
        np.interp(intensity_col, COOL_RANGE, COOL[:, 1]),
        np.interp(intensity_col, COOL_RANGE, COOL[:, 2])]
    
    points = radar_data[:, :-1]
    points[:, :1] = -points[:, :1]
    point_list.points = o3d.utility.Vector3dVector(points)
    point_list.colors = o3d.utility.Vector3dVector(int_color)
    
# Camera callback
def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4)) 

In [35]:
# Set up LIDAR and RADAR, parameters are to assisst visualisation

lidar_bp = bp_lib.find('sensor.lidar.ray_cast') 
lidar_bp.set_attribute('range', '100.0')
lidar_bp.set_attribute('noise_stddev', '0.1')
lidar_bp.set_attribute('upper_fov', '15.0')
lidar_bp.set_attribute('lower_fov', '-25.0')
lidar_bp.set_attribute('channels', '64.0')
lidar_bp.set_attribute('rotation_frequency', '20.0')
lidar_bp.set_attribute('points_per_second', '700000') #default 500000
    
lidar_init_trans = carla.Transform(carla.Location(z=2))
lidar = world.spawn_actor(lidar_bp, lidar_init_trans, attach_to=vehicle)

radar_bp = bp_lib.find('sensor.other.radar') 
radar_bp.set_attribute('horizontal_fov', '30.0') #default 30
radar_bp.set_attribute('vertical_fov', '30.0')   #default 30
radar_bp.set_attribute('points_per_second', '20000') #efault 10000
radar_init_trans = carla.Transform(carla.Location(z=2))
radar = world.spawn_actor(radar_bp, radar_init_trans, attach_to=vehicle)

# avvia camera rgb
camera_bp = bp_lib.find('sensor.camera.rgb') 
camera_init_trans = carla.Transform(carla.Location(z=2.5, x=-3), carla.Rotation())
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

# strutture dati ausiliare per visualizzazione nuvola di punti
point_list = o3d.geometry.PointCloud()
radar_list = o3d.geometry.PointCloud()

# Set up dictionary for camera data
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()
camera_data = {'image': np.zeros((image_h, image_w, 4))} 

# Start sensors
lidar.listen(lambda data: lidar_callback(data, point_list))
radar.listen(lambda data: radar_callback(data, radar_list))
camera.listen(lambda image: camera_callback(image, camera_data))

In [36]:
# OpenCV per telecamera
cv2.namedWindow('RGB Camera', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])
cv2.waitKey(1)


# Open3D visualiser per LIDAR and RADAR
vis = o3d.visualization.Visualizer()
vis.create_window(
    window_name='Carla Lidar',
    width=960,
    height=540,
    left=480,
    top=270)
vis.get_render_option().background_color = [0.05, 0.05, 0.05]
vis.get_render_option().point_size = 1
vis.get_render_option().show_coordinate_frame = True
add_open3d_axis(vis)

# aggiorna la geometria del renderer in base ai punti del lidar e del radar
frame = 0
while True:
    if frame == 2:
        vis.add_geometry(point_list)
        vis.add_geometry(radar_list)
    vis.update_geometry(point_list) #lidar fa una mappatura 3d dell'ambiente circostante alla macchina
    vis.update_geometry(radar_list) #dato un campo visivo, radar rileva le variazioni rilevate durante il movimento. Il blu denota poca o nessuna variazione, verde grosse variazioni
    
    vis.poll_events()
    vis.update_renderer()
    # fix per evitare artefatti grafici
    time.sleep(0.005)
    frame += 1

    cv2.imshow('RGB Camera', camera_data['image'])
    
    # Break if user presses 'q'
    if cv2.waitKey(1) == ord('q'):
        break

# Close displayws and stop sensors
cv2.destroyAllWindows()
radar.stop()
radar.destroy()
lidar.stop()
lidar.destroy()
camera.stop()
camera.destroy()
vis.destroy_window()

for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for actor in world.get_actors().filter('*sensor*'):
    actor.destroy()

C:\Users\Luigi\AppData\Local\Temp\ipykernel_20380\2102998210.py:40: RuntimeWarning: divide by zero encountered in log
  intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004 * 100))


[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
